# Chapter 5. Aggregates and Null

## SQL's Aggregate Operators

    COUNT( * )
    COUNT( [DISTINCT] A)
    SUM( [DISTINCT] A)
    AVG( [DISTINCT] A)
    MAX( A )
    MIN( A )
where A is a single column

* Significant extension of relational algebra
* Computation and summarization operations
* Appears in _target-list_ of query
* Results _aggregate_ rather than appear individually


## Setup for live examples

This document is a Jupyter Notebook (or a static HTML page generated from it). This way we can try live examples in class and you can play with them afterward.

First we'll do some setup of the python environment.

In [35]:
from sqlite3 import connect


### Construct some tables

In [36]:
conn = connect(":memory:")
cursor = conn.cursor()
cursor.execute("""
create table Sailors 
    (sid integer primary key, 
     sname text, 
     rating integer, 
     age real)
""")
sailors = [(22, 'Dustin', 7, 45.0),
           (29, 'Brutus', 1, 33.0),
           (31, 'Lubber', 8, 55.5),
           (32, 'Andy', 8, 25.5),
           (58, 'Rusty', 10, 35.0),
           (64, 'Horatio', 7, 35.0),
           (71, 'Zorba', 10, 16.0),
           (74, 'Horatio', 9, 35.0),
           (85, 'Art', 3, 25.5),
           (95, 'Bob', 3, 63.5),
           (96, 'Frodo', 3, 25.5),
           (99, 'Bob jr.', 3, 63.5),
           ]
for sailor in sailors:
    cursor.execute("insert into Sailors values(?, ?, ?, ?)", sailor)

cursor.execute("""
create table Boats
    (bid integer primary key, 
     bname text, 
     color text)
""")
boats = [(101, "Interlake", "blue"),
         (102, "Interlake", "red"),
         (103, "Clipper", "green"),
         (104, "Marine", "red")]
for boat in boats:
    cursor.execute("insert into Boats values (?, ?, ?)", boat)

cursor.execute("""
create table Reserves 
    (sid integer, 
     bid integer, 
     day text, 
     primary key (sid, bid, day) 
     foreign key (sid) references Sailors
     foreign key (bid) references Boats)
""")
reserves = [(22, 101, '10/10/98'),
            (22, 102, '10/10/98'),
            (22, 103, '10/8/98'),
            (22, 104, '10/7/98'),
            (31, 102, '11/10/98'),
            (31, 103, '11/6/98'),
            (64, 101, '9/5/98'),
            (64, 102, '9/8/98'),
            (74, 103, '9/8/98')]
for reserve in reserves:
    cursor.execute("insert into Reserves values(?, ?, ?)", reserve)


## Examples

How many instances in the Sailors relation?

In [37]:
cursor.execute("""
select count(*) from Sailors S
""")
#for row in cursor:
#    print row
f = cursor.fetchone()
q1 = f[0]
print(q1)


12


Average age of Sailors with a rating of 10?

In [38]:
cursor.execute("""
select avg(S.age) 
    from Sailors S
    where S.rating=10
""")
for row in cursor:
    print(row)

(25.5,)


Names of all Sailors who have achieved the maximum rating.

In [39]:
cursor.execute("""
select S.sname from Sailors S
    where S.rating=(select max(S2.rating) from Sailors S2)
""")
for row in cursor:
    print(row)

('Rusty',)
('Zorba',)


How many distinct ratings for Sailors less than 40 years old?

In [40]:
cursor.execute("""
select count(distinct S.rating)
    from Sailors S
    where S.age < 40.0
""")
for row in cursor:
    print(row)

(6,)


Look at the query above. We didn't count them all, we counted the distinct ones.

In [41]:
cursor.execute("""
select  S.rating
    from Sailors S
    where S.age < 40.0
""")
for row in cursor:
    print(row)

(1,)
(8,)
(10,)
(7,)
(10,)
(9,)
(3,)
(3,)


How many reservations were made by Sailors less than 40 years old?

In [42]:
cursor.execute("""
select count(*)
    from Sailors S, Reserves R
    where S.sid = R.sid and S.age < 40
""")
for row in cursor:
    print(row)

(3,)


## Find name and age of the oldest Sailors

This first version is invalid in SQL because anything **not** aggregated in the _select-list_ must be named in the _grouping-list_. sqlite appears to accept it. But its interpretation is broken because it doesn't handle multiple sailors with the same age; it is reporting one that it saw and you can't be sure which.

In [43]:
cursor.execute("""
select S.sname, max(S.age) from Sailors S
""")
for row in cursor:
    print(row)

('Bob', 63.5)


In [44]:
cursor.execute("""
select S.sname, S.age
    from Sailors S
    where S.age = 
        (select max(S2.age)
         from Sailors S2)
""")
for row in cursor:
    print(row)

('Bob', 63.5)
('Bob jr.', 63.5)


## Motivation for Grouping

* So far, we've applied aggregate operators to _all_ (qualifying) tuples. Sometimes, we want to apply them to _subgroups_.
* Consider: _Find the age of the youngest sailor for each rating level_
  + In general, we don't know hay many rating levels exist, and what they are.
  + Suppose we know that rating values go from 1 to 10; we **could** write 10 queries like this:

In [45]:
for i in range(1, 11):
    cursor.execute("""
    select min(S.age)
        from Sailors S
        where S.rating=?
    """, (i,))
    for row in cursor:
        print(i, row)

1 (33.0,)
2 (None,)
3 (25.5,)
4 (None,)
5 (None,)
6 (None,)
7 (35.0,)
8 (25.5,)
9 (35.0,)
10 (16.0,)


Instead we can use **group by** and **having**

## Queries with group by and having

**select** [**distinct**] _target-list_
**from** _relation-list_
**where** _qualification_
**group by** _grouping-list_
**having** _group-qualification_

* The _target-list_ contains
 1. _attribute-list_
 2. terms with aggregate operations
* The _attribute-list_ (1) must be a subset of _grouping-list_. Each answer tuple corresponds to a _group_, and the attributes must have a single value per group. (A _group_ is a set of tuples that have the same value for all attributes in _grouping-list_.)

## Conceptual Evaluation
+ The cross-product of _relation-list_ is computed
+ tuples that fail qualification are discarded
+ unnecessary fields are deleted,
+ the remaining tuples are partitioned into groups by the value of attributes in grouping-list
+ The _group-qualification_ is then applied to eliminate some groups. Expressions in _group-qualification_ must have a single value per group. In effect, an attribute in _group-qualification_ that is not an argument to an aggregate op also appears in _grouping-list_. (SQL does not exploit primary key semantics here)
+ One answer tuple is generated per qualifying group.


## Group by examples

### Find the age of the youngest sailor with age >= 18 for each rating with at least 2 such sailors

In [46]:
cursor.execute("""
select S.rating, min(S.age) as minage
    from Sailors S
    where S.age >= 18
    group by S.rating
    having count(*) > 1
""")
for row in cursor:
    print(row)

(3, 25.5)
(7, 35.0)
(8, 25.5)


#### Let's break that down into steps to see what is happening.

<div style="width:100%;overflow:auto">

<div style="width:25%;float:left">
Select age and rating.
<table>
    <tr><th>rating</th><th>age</th></tr>
    <tr><td>7</td><td>45.0</td></tr>
    <tr><td>1</td><td>33.0</td></tr>
    <tr><td>8</td><td>55.5</td></tr>
    <tr><td>8</td><td>25.5</td></tr>
    <tr><td>10</td><td>35.0</td></tr>
    <tr><td>7</td><td>35.0</td></tr>
    <tr><td>10</td><td>16.0</td></tr>
    <tr><td>9</td><td>35.0</td></tr>
    <tr><td>3</td><td>25.5</td></tr>
    <tr><td>3</td><td>63.5</td></tr>
    <tr><td>3</td><td>25.5</td></tr>
</table>
</div>
<div style="width:25%;float:left">


Apply where >= 18.

<table>
    <tr><th>rating</th><th>age</th></tr>
    <tr><td>1</td><td>33.0</td></tr>
    <tr><td>3</td><td>25.5</td></tr>
    <tr><td>3</td><td>63.5</td></tr>
    <tr><td>3</td><td>25.5</td></tr>
    <tr><td>7</td><td>45.0</td></tr>
    <tr><td>7</td><td>35.0</td></tr>
    <tr><td>8</td><td>55.5</td></tr>
    <tr><td>8</td><td>25.5</td></tr>
    <tr><td>9</td><td>35.0</td></tr>
    <tr><td>10</td><td>35.0</td></tr>
</table>
</div>
<div style="width:25%;float:left">

Eliminate groups < 2

<table>
    <tr><th>rating</th><th>age</th></tr>
    <tr><td>3</td><td>25.5</td></tr>
    <tr><td>3</td><td>63.5</td></tr>
    <tr><td>3</td><td>25.5</td></tr>
    <tr><td>7</td><td>45.0</td></tr>
    <tr><td>7</td><td>35.0</td></tr>
    <tr><td>8</td><td>55.5</td></tr>
    <tr><td>8</td><td>25.5</td></tr>
</table>
</div>

<div style="width:25%;float:left">

Aggregate over each group.

<table>
    <tr><th>rating</th><th>age</th></tr>
    <tr><td>3</td><td>25.5</td></tr>
    <tr><td>7</td><td>35.0</td></tr>
    <tr><td>8</td><td>25.5</td></tr>
</table>
</div>



### Find the age of the youngest Sailor with age >= 18, for each rating with at least 2 sailors between 18 and 60.

In [47]:
cursor.execute("""
select S.rating, min(S.age)
    from Sailors S
    where S.age >= 18 and S.age <= 60
    group by S.rating
    having count(*) > 1
""")
for row in cursor:
    print(row)

(3, 25.5)
(7, 35.0)
(8, 25.5)


I can name an aggregate and use it in other expressions.

In [48]:
# random example with minage
cursor.execute("""
select S.rating, min(S.age) as minage
    from Sailors S
    where S.age <= 60
    group by S.rating
    having minage >= 20
""")
for row in cursor:
    print(row)

(1, 33.0)
(3, 25.5)
(7, 35.0)
(8, 25.5)
(9, 35.0)


### Find the number of reservations for each red boat

In [49]:
cursor.execute("""
select B.bid, count(*) as scount
    from Sailors S, Boats B, Reserves R
    where S.sid=R.sid and R.bid=B.bid and B.color='red'
    group by B.bid
""")
for row in cursor:
    print(row)

(102, 3)
(104, 1)


Above, we are grouping over a join of three relations

What do we get if we remove B.color='red' from the **where** clause and add a **having** clause with this condition?

In [50]:
cursor.execute("""
select B.bid, count(*) as scount
    from Sailors S, Boats B, Reserves R
    where S.sid=R.sid and R.bid=B.bid
    group by B.bid
    having B.color='red'
""")
for row in cursor:
    print(row)

(102, 3)
(104, 1)


What if we drop Sailors and the condition involving S.sid?

In [51]:
cursor.execute("""
select B.bid, count(*) as scount
    from Boats B, Reserves R
    where R.bid=B.bid
    group by B.bid
    having B.color='red'
""")
for row in cursor:
    print(row)

(102, 3)
(104, 1)


### Find age of the youngest sailor 18 or older for each rating with at least 2 sailors (of any age).

In [52]:
cursor.execute("""
select S.rating, min(S.age)
    from Sailors S
    where S.age >= 18
    group by S.rating
    having 1 < (select count(*)
                    from Sailors S2
                    where S.rating=S2.rating)
""")
for row in cursor:
    print(row)

(3, 25.5)
(7, 35.0)
(8, 25.5)
(10, 35.0)


Above shows **having** clause can also contain a subquery.

Compare this with the query where we considered only ratings with 2 sailors over 18.

What if **having** clause is replaced by **having count**(*) > 1

In [53]:
cursor.execute("""
select S.rating, min(S.age)
    from Sailors S
    where S.age >= 18
    group by S.rating
    having count(*) > 1
""")
for row in cursor:
    print(row)

(3, 25.5)
(7, 35.0)
(8, 25.5)


Why the different answer?

### Find those ratings for which the average age is the minimum over all ratings

This first attempt won't work because aggregate operations cannot be nested.

In [54]:
cursor.execute("""
select S.rating
    from Sailors S
    where S.age = (select min(avg(S2.age)) from Sailors S2)
""")
for row in cursor:
    print(row)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: misuse of aggregate function avg()

This next one doesn't work either. Perhaps an sqlite limitation?

In [55]:
cursor.execute("""
select Temp.rating, Temp.avgage
from (select S.rating, avg(S.age) as avgage
      from Sailors S
      group by S.rating) as Temp
where Temp.avgage = (select min(Temp.avgage) from Temp)
""")
for row in cursor:
    print(row)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: no such table: Temp

I can create a temporary table myself. 

In [56]:
# drop the temporary table if it already exists to allow me to run
# this more than once
cursor.execute("""
drop table if exists Temp
""");
# create the temporary table
cursor.execute("""
create temp table Temp as
    select S.rating as rating, avg(S.age) as avgage
        from Sailors S
        group by S.rating
""")
# show its content
cursor.execute("""
select * from Temp
""")
print('temp table')
for row in cursor:
    print(row)
# now do the query on the temporary table
cursor.execute("""
select T.rating, T.avgage
    from Temp T
    where T.avgage = (select min(Temp.avgage) from Temp)
""")
print('final result')
for row in cursor:
    print(row)

temp table
(1, 33.0)
(3, 44.5)
(7, 40.0)
(8, 40.5)
(9, 35.0)
(10, 25.5)
final result
(10, 25.5)


## Null Values

* Field values in a tuple are sometimes unknown (e.g., a rating has not been assigned) or _inapplicable_ (e.g., no spouse's name).
* SQL provides a special value _null_ for such situations.
* The presence of _null_ complicates many issues.
 + What happens if we compare with _null_?
 + Special operators need to check if value is [not] null
 + We need a _3-valued logic_ (true, false and unknown)
 + Meaning of constructs must be defined carefully (e.g. **where** eliminates rows that don't evaluate to true.)
 + New operators (e.g. _outer joins_) needed.

In [57]:
# a place to play, try replacing the operator
cursor.execute("""
select 1 = 1 and 1 < null;
""")
for row in cursor:
    print(row)

(None,)


SQL introduces a variety of joins that take advantage of _null_ values. In a normal join, Sailors that do not match some row in Reserves according to the join condition do not appear in the result. In an **outer join** Sailors without a matching Reserves row appear once in the result with the columns inherited from Reserves set to _null_ values.

In [58]:
cursor.execute("""
select S.sid, R.bid
    from Sailors S natural left outer join Reserves R
""")
for row in cursor:
    print(row)

(22, 101)
(22, 102)
(22, 103)
(22, 104)
(29, None)
(31, 102)
(31, 103)
(32, None)
(58, None)
(64, 101)
(64, 102)
(71, None)
(74, 103)
(85, None)
(95, None)
(96, None)
(99, None)


We can disallow _null_ values by specifying **not null** as part of the field definition. **primary key** fields are prohibitied from having _null_ values.